In [18]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import tensorflow as tf
from typing import Tuple

Now that we understand how to get the data and model, we're ready to train the neural network. First, we need to load the data using the technique we discussed in the first unit. In order not to clutter this notebook, we've added the `get_data` function and `NeuralNetwork` class you've already seen to a separate `kintro.py` file, which we'll import here. 

In [3]:
!wget -Nq https://raw.githubusercontent.com/MicrosoftDocs/tensorflow-learning-path/main/intro-keras/kintro.py
from kintro import *

In order to understand what happens during training, we need to add a little more detail to our neural network visualization:

![Basic neural network with details](notebooks/images/2-basic-nn-with-details.png)

Notice that we've added weights $W$ to the connections between layers, and bias $b$ as input to `Dense` layers &mdash; $W$ and $b$ are the neural network's parameters. Our goal when training our network (also known as fitting) is to find the parameters $W$ and $b$ that minimize the differences between the actual and predicted labels for our data. In order to do this, we add a loss function at the end of our network, which compares the actual and predicted values, returning a large number if they're different and a smaller number if they're similar.

The loss function that is most frequently used for classification is cross-entropy loss, and Keras provides several varieties of this loss. Because our dataset includes a single integer label for each example, we use the [`SparseCategoricalCrossentropy`](https://www.tensorflow.org/api_docs/python/tf/keras/losses/SparseCategoricalCrossentropy) loss function.

When we introduce a `Dense` layer, a linear operation involving the input data and parameters $W$ and $b$ is performed. For example, the top node of the first linear layer performs the following calculation:

$$
z^1_1 = w^1_{1,1} x_1 + ... + w^1_{1,784} x_{784} + b^1_1
$$

If we specify a ReLU **activation function**, the output of the linear operation is then passed as input to a ReLU function:

$$
a^1_1 = ReLU(z^1_1)
$$

Mathematically speaking, we can now think of our neural network as a function $\ell$ that takes as input the data $X$, expected labels $y$, and parameters $W$ and $b$, then performs a sequence of operations on that data, and returns a loss. 

$$
\mathrm{loss} = \ell(X, y, W, b)
$$

Our goal is to find the parameters $W$ and $b$ that lead to the lowest possible loss. (We can't change our data $X$ or the corresponding labels $y$ &mdash; they're fixed &mdash; but we can adjust $W$ and $b$.) It turns out that problems of this kind fall in the well-studied mathematical area of optimization. The simplest minimization algorithm is gradient descent, and in this sample we use a variation known as Stochastic Gradient Descent or [`SGD`](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/SGD). 

We're now ready to "compile" the model &mdash; this is where we tell it that we want to use the `SGD` optimizer and the `SparseCategoricalCrossentropy` loss function. We also tell the model that we want it to report on the accuracy during training. 

In [6]:
learning_rate = 0.1
batch_size = 64

(train_dataset, test_dataset) = get_data(batch_size)

model = NeuralNetwork()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['accuracy']
model.compile(optimizer, loss_fn, metrics)

A few details from the code above deserve a quick explanation. 

Notice that we pass `from_logits=True` to the loss function. This is because the categorical cross-entropy function requires a probability distribution as input, meaning that the numbers should be between zero and one, and they should add up to one. Our network produces a vector of numbers that have no upper or lower bound (called "logits"), so we need to normalize them to get a probability distribution. This is typically done using the `softmax` function, and specifying `from_logits=True` automatically calculates the softmax before computing the loss.

Notice also that we pass a `learning_rate` to the `SGD` optimizer. The learning rate is a parameter needed in the gradient descent algorithm. We could have left it at the default, which is 0.01, but it's important to know how to specify it because different learning rates can lead to very different prediction accuracies.

Finally, notice that we specified a `batch_size`, which we used in the construction of the `Dataset`, as we saw earlier. This is important during training, because it tells the model that we want to train on 64 images at a time. You might be wondering why 64? Why not train on a single image at a time? Or all 60,000 images at once? Doing a complete training step for each individual image would be inefficient because we would have to perform all the calculations 60,000 times in order to account for every input image. If we included all the input images in $X$, we'd need a lot of memory, and we'd spend a lot of time computing each training step. So we settle for a size in between, called the "mini-batch" size. 

Now that we've configured our model with the parameters we need for training, we can call `fit` to train the model. We specify the number of epochs as 2, which means that we want to iterate over the complete set of 60,000 training images twice while training the neural network. We use just two epochs in this sample because we don't want to spend too much time training the network in a demo. In a real project, you will want to increase the number of epochs, which will lead to better accuracy of the resulting model.

In [7]:
epochs = 2
print('\nFitting:')
model.fit(train_dataset, epochs=epochs)


Fitting:
Epoch 1/2
938/938 [==============================] - 7s 2ms/step - loss: 0.5589 - accuracy: 0.7991
Epoch 2/2
938/938 [==============================] - 2s 2ms/step - loss: 0.3956 - accuracy: 0.8547


Training has found values for the parameters $W$ and $b$ such that, when we provide an image as input, we'll get a reasonable prediction as output. Our model is now ready to be tested. Remember that when we loaded the data, we obtained two datasets, one with training data and another with test data. It's time to use the test dataset.

In [8]:
print('\nEvaluating:')
(test_loss, test_accuracy) = model.evaluate(test_dataset)
print(f'\nTest accuracy: {test_accuracy * 100:>0.1f}%, test loss: {test_loss:>8f}')


Evaluating:
157/157 [==============================] - 1s 2ms/step - loss: 0.4100 - accuracy: 0.8482

Test accuracy: 84.8%, test loss: 0.409994


We've achieved pretty good test accuracy, considering that we used such a simple network and only two epochs of training. We're done with training and can now save the parameters $W$ and $b$ that were calculated.

In [9]:
model.save_weights('outputs/weights')

Now that our neural network has appropriate values for its parameters, we can use it to make a prediction.